In [5]:
from imports import *
from utils import *
from model import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading

dataset = 'ECG'

# 使用函数创建数据集
data_dir = 'raw\ECG'
train_dataset, test_dataset = load_ecg_data(data_dir)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# Model Definition

# Define the number of classes for the classification task
num_classes = 2

# Initialize the model with the given configuration and number of classes, and move it to the specified device
model = NET_4k_ECG(num_classes).to(device)

total_weights = count_weights(model)
print(f"The network has {total_weights} weights that need to be deployed.")

model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
model = torch.quantization.prepare_qat(model, inplace=True)
model = torch.quantization.convert(model, inplace=True)

The network has 317 weights that need to be deployed.


In [7]:
# Load Pretrained Model

# Specify the filename of the saved model
model_filename = f"checkpoint/ECG-317-99.30_model.t7"

# Load the checkpoint from the file, mapping the model to the specified device
checkpoint = torch.load(model_filename, map_location=device)

# Load the model state dictionary from the checkpoint
model.load_state_dict(checkpoint['model'])

print(model) # The model has been quantized to int8

NET_4k_ECG(
  (quant): Quantize(scale=tensor([0.0372]), zero_point=tensor([46]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (conv1_depthwise): QuantizedConv1d(1, 1, kernel_size=(7,), stride=(1,), scale=0.05332726612687111, zero_point=44, padding=(4,), dilation=(2,))
  (conv1_pointwise): QuantizedConv1d(1, 10, kernel_size=(1,), stride=(1,), scale=0.08095549792051315, zero_point=55)
  (bn1): QuantizedBatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): QuantizedHardswish()
  (conv2_depthwise): QuantizedConv1d(10, 10, kernel_size=(5,), stride=(1,), scale=0.12382008880376816, zero_point=66, padding=(1,), groups=10)
  (conv2_pointwise): QuantizedConv1d(10, 10, kernel_size=(1,), stride=(1,), scale=0.26861637830734253, zero_point=52)
  (activation2): QuantizedHardswish()
  (conv3_depthwise): QuantizedConv1d(10, 10, kernel_size=(3,), stride=(1,), scale=0.1977425515651703, zero_point=51, padding=(1,), groups=10)
  (conv3_pointwise): QuantizedConv

In [8]:
# Test the model on the test dataset and store the accuracy
model.eval()
test_accuracy = test_model(model, test_loader)

# Extract the int8 weights from all quantized layers in the model and save them to a list
quantized_weights_list = print_quantized_weights(model)

print(f"The test accuracy after quantization is: {test_accuracy:.2f}%")

Extracting weights from layer: conv1_depthwise
Weight Shape: (7,), Int8 Weights:
[-89 127  68  24  -3  -3  -6]

Extracting weights from layer: conv1_pointwise
Weight Shape: (10,), Int8 Weights:
[-127 -128  127  127 -128  127 -128 -128  127 -128]

Extracting weights from layer: conv2_depthwise
Weight Shape: (50,), Int8 Weights:
[ -16  -34 -128  -98  -17   42 -127    1  -36    8  127   26  -47  -33
 -106  -11  -85  -35  127   -7  -46   29 -128   37  -88  127  -31  -48
  -49   30   64  -36  119  127  115   35 -104  -43  127   43 -128  -12
    7   11   23   38   22   84  117  127]

Extracting weights from layer: conv2_pointwise
Weight Shape: (100,), Int8 Weights:
[ -11  -34  -70 -128   76  -62  -54    6   24   25  -68   11   37  127
  -54  -61   44   85   87   63   -3  105  127  111  109    6   32  114
  -97  -36   22  -16  -28   81  -83 -128   15   53  -22  -21    3   57
   50  -55   34   13  127  -69  -10   47  -47   33   69  -52  -15    8
  127   10  -24    6   -7  -80  117   19   86   